In [1]:
import os
import numpy as np
import glob
from matplotlib import pyplot as plt

In [2]:
import rasterio
import earthpy.plot as ep
from patchify import patchify
import tifffile as tifffile
import random

In [3]:
# check working directory
os.getcwd()

'/data/private/Thesis/May'

### **Patching Single Datasets**

In [4]:
# read val image

with rasterio.open("Data/02_Tile/label_bin_gaveau/label_bin_gaveau11.tif") as src:
    lbl = src.read()

print(lbl.shape)

(1, 5903, 6055)


In [5]:
lbl = np.transpose(lbl, (1,2,0))

In [6]:
PATCH_SIZE = 256
N_BANDS_LBL = lbl.shape[-1]

print(N_BANDS_LBL)

1


In [7]:
patches_msk = patchify(lbl, (PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL), step=PATCH_SIZE)

if not os.path.exists('Valmsk'):
    os.makedirs('Valmsk')

for i in range(patches_msk.shape[0]):
    for j in range(patches_msk.shape[1]):
        
        single_patch_msk = patches_msk[i,j,:,:]
        tifffile.imwrite('Valmsk/' + 'valmsk_' + str(i) + '_' + str(j) + ".tif", single_patch_msk)

In [8]:
msk_patch = sorted(glob.glob('Valmsk/*.tif'))

print(len(msk_patch))

529


In [9]:
y = np.zeros((len(msk_patch), PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL))

for i, (msk) in enumerate(zip(msk_patch)):
    msk = tifffile.imread(msk)
    np_msk = np.array(msk)
    y[i] = np_msk

In [10]:
print(y.shape)

(529, 256, 256, 1)


In [11]:
if not os.path.exists('Data/03_Patch/val'):
    os.makedirs('Data/03_Patch/val')

np.save('Data/03_Patch/val/ygaveau.npy', y)

### **Patching Validation Datasets**

In [4]:
# read val image

with rasterio.open("Data/02_Tile/s2/s211.tif") as src:
    s2 = src.read()
with rasterio.open("Data/02_Tile/s1/s111.tif") as src:
    s1 = src.read()
with rasterio.open("Data/02_Tile/label_bin_descals/label_bin_descals11.tif") as src:
    lbl = src.read()

print(s1.shape)
print(s2.shape)
print(lbl.shape)

(2, 5903, 6055)
(4, 5903, 6055)
(1, 5903, 6055)


In [5]:
s1 = np.transpose(s1, (1,2,0))
s2 = np.transpose(s2, (1,2,0))
lbl = np.transpose(lbl, (1,2,0))

print(s1.shape)
print(s2.shape)
print(lbl.shape)

(5903, 6055, 2)
(5903, 6055, 4)
(5903, 6055, 1)


In [6]:
# define criterias
PATCH_SIZE = 256
N_BANDS_S1 = s1.shape[-1]
N_BANDS_S2 = s2.shape[-1]
N_BANDS_LBL = lbl.shape[-1]

print(N_BANDS_S1)
print(N_BANDS_S2)
print(N_BANDS_LBL)

2
4
1


In [7]:
# Create patch for validation
patches_img1 = patchify(s1, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S1), step=PATCH_SIZE)

if not os.path.exists('Vals1'):
    os.makedirs('Vals1')

for i in range(patches_img1.shape[0]):
    for j in range(patches_img1.shape[1]):
        
        single_patch_img1 = patches_img1[i,j,:,:]
        tifffile.imwrite('Vals1/' + 'vals1_' + str(i) + '_' + str(j) + ".tif", single_patch_img1)

# Create patch for validation
patches_img2 = patchify(s2, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S2), step=PATCH_SIZE)

if not os.path.exists('Vals2'):
    os.makedirs('Vals2')

for i in range(patches_img2.shape[0]):
    for j in range(patches_img2.shape[1]):
        
        single_patch_img2 = patches_img2[i,j,:,:]
        tifffile.imwrite('Vals2/' + 'vals2_' + str(i) + '_' + str(j) + ".tif", single_patch_img2)

# Create patch for validation
patches_msk = patchify(lbl, (PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL), step=PATCH_SIZE)

if not os.path.exists('Valmsk'):
    os.makedirs('Valmsk')

for i in range(patches_msk.shape[0]):
    for j in range(patches_msk.shape[1]):
        
        single_patch_msk = patches_msk[i,j,:,:]
        tifffile.imwrite('Valmsk/' + 'valmsk_' + str(i) + '_' + str(j) + ".tif", single_patch_msk)

KeyboardInterrupt: 

In [14]:
# sort and store in numpy format
img_patch1 = sorted(glob.glob('Vals1/*.tif'))
img_patch2 = sorted(glob.glob('Vals2/*.tif'))
msk_patch = sorted(glob.glob('Valmsk/*.tif'))

print(len(img_patch1))
print(len(img_patch2))
print(len(msk_patch))

529
529
529


In [15]:
x1 = np.zeros((len(img_patch1), PATCH_SIZE, PATCH_SIZE, N_BANDS_S1))

for i, (img1) in enumerate(zip(img_patch1)):
    img1 = tifffile.imread(img1)
    np_img1 = np.array(img1)
    x1[i] = np_img1

x2 = np.zeros((len(img_patch2), PATCH_SIZE, PATCH_SIZE, N_BANDS_S2))

for i, (img2) in enumerate(zip(img_patch2)):
    img2 = tifffile.imread(img2)
    np_img2 = np.array(img2)
    x2[i] = np_img2
    
y = np.zeros((len(msk_patch), PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL))

for i, (msk) in enumerate(zip(msk_patch)):
    msk = tifffile.imread(msk)
    np_msk = np.array(msk)
    y[i] = np_msk

In [17]:
if not os.path.exists('Data/03_Patch/val'):
    os.makedirs('Data/03_Patch/val')

np.save('Data/03_Patch/val/x1.npy', x1)
np.save('Data/03_Patch/val/x2.npy', x2)
np.save('Data/03_Patch/val/y.npy', y)

### **Patching Training Datasets**

In [8]:
# read train image
with rasterio.open("Data/02_Tile/s1/s100.tif") as src:
    s100 = src.read()
with rasterio.open("Data/02_Tile/s1/s101.tif") as src:
    s101 = src.read()

# read train image
with rasterio.open("Data/02_Tile/s2/s200.tif") as src:
    s200 = src.read()
with rasterio.open("Data/02_Tile/s2/s201.tif") as src:
    s201 = src.read()

# read train mask
with rasterio.open("Data/02_Tile/label_bin_descals/label_bin_descals00.tif") as src:
    lbl00 = src.read()
with rasterio.open("Data/02_Tile/label_bin_descals/label_bin_descals01.tif") as src:
    lbl01 = src.read()

print(s100.shape)
print(s101.shape)
print(s201.shape)
print(s201.shape)
print(lbl00.shape)
print(lbl01.shape)

(2, 5903, 6055)
(2, 5903, 6055)
(4, 5903, 6055)
(4, 5903, 6055)
(1, 5903, 6055)
(1, 5903, 6055)


In [9]:
# change order of the array, so the n channel is at axis -1 (last axis)
s100 = np.transpose(s100, (1,2,0))
s101 = np.transpose(s101, (1,2,0))
s200 = np.transpose(s200, (1,2,0))
s201 = np.transpose(s201, (1,2,0))
lbl00 = np.transpose(lbl00, (1,2,0))
lbl01 = np.transpose(lbl01, (1,2,0))

print(s100.shape)
print(s101.shape)
print(s201.shape)
print(s201.shape)
print(lbl00.shape)
print(lbl01.shape)

(5903, 6055, 2)
(5903, 6055, 2)
(5903, 6055, 4)
(5903, 6055, 4)
(5903, 6055, 1)
(5903, 6055, 1)


In [10]:
# Create patch for training

# S100 - S101
patches_img00 = patchify(s100, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S1), step=PATCH_SIZE)
patches_img01 = patchify(s101, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S1), step=PATCH_SIZE)

if not os.path.exists('s1'):
    os.makedirs('s1')

for i in range(patches_img00.shape[0]):
    for j in range(patches_img00.shape[1]):
        
        single_patch_img = patches_img00[i,j,:,:]
        tifffile.imwrite('s1/' + 's100_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

for i in range(patches_img01.shape[0]):
    for j in range(patches_img01.shape[1]):
        
        single_patch_img = patches_img01[i,j,:,:]
        tifffile.imwrite('s1/' + 's101_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

# Create patch for validation
patches_img00 = patchify(s200, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S2), step=PATCH_SIZE)
patches_img01 = patchify(s201, (PATCH_SIZE, PATCH_SIZE, N_BANDS_S2), step=PATCH_SIZE)

if not os.path.exists('s2'):
    os.makedirs('s2')

for i in range(patches_img00.shape[0]):
    for j in range(patches_img00.shape[1]):
        
        single_patch_img = patches_img00[i,j,:,:]
        tifffile.imwrite('s2/' + 's200_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

for i in range(patches_img01.shape[0]):
    for j in range(patches_img01.shape[1]):
        
        single_patch_img = patches_img01[i,j,:,:]
        tifffile.imwrite('s2/' + 's201_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

# Create patch for validation
patches_img00 = patchify(lbl00, (PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL), step=PATCH_SIZE)
patches_img01 = patchify(lbl01, (PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL), step=PATCH_SIZE)

if not os.path.exists('msk'):
    os.makedirs('msk')

for i in range(patches_img00.shape[0]):
    for j in range(patches_img00.shape[1]):
        
        single_patch_img = patches_img00[i,j,:,:]
        tifffile.imwrite('msk/' + 'msk00_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

for i in range(patches_img01.shape[0]):
    for j in range(patches_img01.shape[1]):
        
        single_patch_img = patches_img01[i,j,:,:]
        tifffile.imwrite('msk/' + 'msk01_' + str(i) + '_' + str(j) + ".tif", single_patch_img)

In [11]:
# sort and store in numpy format
img_patch1 = sorted(glob.glob('s1/*.tif'))
img_patch2 = sorted(glob.glob('s2/*.tif'))
msk_patch = sorted(glob.glob('msk/*.tif'))

print(len(img_patch1))
print(len(img_patch2))
print(len(msk_patch))

1058
1058
1058


In [13]:
x1 = np.zeros((len(img_patch1), PATCH_SIZE, PATCH_SIZE, N_BANDS_S1))

for i, (img1) in enumerate(zip(img_patch1)):
    img1 = tifffile.imread(img1)
    np_img1 = np.array(img1)
    x1[i] = np_img1

x2 = np.zeros((len(img_patch2), PATCH_SIZE, PATCH_SIZE, N_BANDS_S2))

for i, (img2) in enumerate(zip(img_patch2)):
    img2 = tifffile.imread(img2)
    np_img2 = np.array(img2)
    x2[i] = np_img2
    
y = np.zeros((len(msk_patch), PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL))

for i, (msk) in enumerate(zip(msk_patch)):
    msk = tifffile.imread(msk)
    np_msk = np.array(msk)
    y[i] = np_msk

In [14]:
print(x1.shape)
print(x2.shape)
print(y.shape)

(1058, 256, 256, 2)
(1058, 256, 256, 4)
(1058, 256, 256, 1)


In [15]:
if not os.path.exists('Data/03_Patch/train'):
    os.makedirs('Data/03_Patch/train')

np.save('Data/03_Patch/train/x1.npy', x1)
np.save('Data/03_Patch/train/x2.npy', x2)
np.save('Data/03_Patch/train/y.npy', y)